In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
from dedalus.extras.plot_tools import plot_bot_2d
from eigentools import Eigenproblem, CriticalFinder
figkw = {'figsize':(6,4), 'dpi':100}

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
def orr_sommerfeld_problem(R):
    # Basis
    xbasis = d3.Chebyshev('x', 128, interval=(-1,1), dealias=2)
    domain = d3.Domain([xbasis], grid_dtype=np.complex128)

    # Problem
    problem = d3.EVP(domain, ['u','u_x','u_xx','u_xxx'], eigenvalue='lamb')
    problem.parameters['R'] = R
    problem.add_equation("(1/R)*(dx(u_xxx) - 2*dx(u_x) + u) - 2j*u - 1j*(1-x**2)*(dx(u_x)-u) - lamb*(dx(u_x) - u) = 0")
    problem.add_equation("u_xxx - dx(u_xx) = 0")
    problem.add_equation("u_xx - dx(u_x) = 0")
    problem.add_equation("u_x-dx(u) = 0")
    problem.add_bc("left(u)=0")
    problem.add_bc("right(u)=0")
    problem.add_bc("left(u_x)=0")
    problem.add_bc("right(u_x)=0")

    return problem

In [10]:
eigvals = [None]*200
i = 0
for R in np.arange(5770,5774,0.1):
    problem = orr_sommerfeld_problem(R)
    EP = Eigenproblem(problem)
    EP.solve(sparse=False)
    eigvals[i] = EP.evalues
    i = i+1

2022-10-18 16:59:07,497 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 16:59:07,512 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 16:59:22,136 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 16:59:22,151 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 16:59:36,213 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 16:59:36,231 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 16:59:50,206 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 16:59:50,216 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 17:00:03,944 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 17:00:03,968 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 17:00:18,451 proble

# Potentially need to offset eigenvalues by the "zero-point" eigenvalue for validity

In [96]:
max_eigvals = list(filter(lambda x: x, [np.max(np.real(evs)) for evs in eigvals]))
max_eigvals - np.max(max_eigvals[22])

array([-4.05256182e-06, -3.86827675e-06, -3.68399911e-06, -3.49972887e-06,
       -3.31546601e-06, -3.13121058e-06, -2.94696251e-06, -2.76272187e-06,
       -2.57848863e-06, -2.39426278e-06, -2.21004434e-06, -2.02583329e-06,
       -1.84162964e-06, -1.65743339e-06, -1.47324454e-06, -1.28906308e-06,
       -1.10488903e-06, -9.20722370e-07, -7.36563099e-07, -5.52411229e-07,
       -3.68266760e-07, -1.84129688e-07,  0.00000000e+00,  1.84122285e-07,
        3.68237169e-07,  5.52344679e-07,  7.36444772e-07,  9.20537485e-07,
        1.10462280e-06,  1.28870073e-06,  1.47277125e-06,  1.65683440e-06,
        1.84089014e-06,  2.02493850e-06,  2.20897947e-06,  2.39301304e-06,
        2.57703922e-06,  2.76105801e-06,  2.94506941e-06,  3.12907343e-06])

In [118]:
max_eigvals

[-8.187572542697629e-05,
 -8.16914403565943e-05,
 -8.150716271724294e-05,
 -8.132289247131397e-05,
 -8.113862961321146e-05,
 -8.095437417975542e-05,
 -8.077012611380282e-05,
 -8.058588547656772e-05,
 -8.040165223722149e-05,
 -8.021742638561628e-05,
 -8.003320794056021e-05,
 -7.984899689092838e-05,
 -7.966479323954623e-05,
 -7.948059699382729e-05,
 -7.929640814130731e-05,
 -7.911222668699673e-05,
 -7.892805263280111e-05,
 -7.874388597413289e-05,
 -7.855972670312609e-05,
 -7.837557483335066e-05,
 -7.819143036469987e-05,
 -7.800729329228292e-05,
 -7.782316360439452e-05,
 -7.763904131982959e-05,
 -7.745492643546374e-05,
 -7.727081892552689e-05,
 -7.708671883198351e-05,
 -7.690262611914864e-05,
 -7.67185408047792e-05,
 -7.653446287258929e-05,
 -7.635039235032035e-05,
 -7.616632920363537e-05,
 -7.598227346614864e-05,
 -7.579822510514947e-05,
 -7.561418413884127e-05,
 -7.543015056664346e-05,
 -7.524612438624953e-05,
 -7.506210558999943e-05,
 -7.487809419377685e-05,
 -7.469409017092457e-05]

# Why does running the same code give two different answers?
It seems like the "number of problems" is varying between these two code blocks -- what does that do? What determines these problem numbers?

In [106]:
ep = Eigenproblem(orr_sommerfeld_problem(5722.22))
ep.solve(sparse=False)
np.max(np.real(ep.evalues))

2022-10-18 18:23:14,834 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 18:23:14,850 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10


-0.00017078022071437827

In [112]:
ep2__ = Eigenproblem(orr_sommerfeld_problem(5772.22))
s = ep2__.solve(sparse=False)
np.max(np.real(ep2__.evalues))

2022-10-18 18:24:14,133 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-18 18:24:14,147 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10


-7.778633855524981e-05

In [119]:
ep3 = Eigenproblem(orr_sommerfeld_problem(6000))
ep3.solve(sparse=False)
np.max(np.real(ep3.evalues))

2022-10-19 16:03:50,022 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2022-10-19 16:03:50,076 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10


0.0003230886753543836

(-7.778633855524981e-05, 62, -0.26156594282372503)

## Try with $k=1.02$